In [1]:
import os
import numpy as np
import pandas as pd
import PIL.Image as Image

import torch 
import torch.nn as nn
import torch.functional as F

from torchvision.transforms import Compose, ToTensor, Resize




In [2]:
test_image = os.path.join('..','data','down-stream','binary','Normal','1.jpg')

In [3]:
test_image = Image.open(test_image)

In [4]:
transforms = Compose([Resize((224,224)),
    ToTensor()
    ])

In [5]:
test_image = transforms(test_image).unsqueeze(0)
test_image.shape

torch.Size([1, 3, 224, 224])

In [6]:
class Bottleneck(nn.Module):
    def __init__(self,in_channels, growth_rate):
        super(Bottleneck,self).__init__()
        
        inner_channels = 4 * growth_rate
        
        
        self.bottleneck = nn.Sequential(nn.BatchNorm2d(in_channels),
                                        nn.ReLU(inplace= True),
                                        nn.Conv2d(in_channels= in_channels, out_channels= inner_channels,
                                                  kernel_size= 1, bias= False),
                                        nn.BatchNorm2d(inner_channels),
                                        nn.ReLU(inplace= True),
                                        nn.Conv2d(in_channels= inner_channels, out_channels= growth_rate,
                                                  kernel_size= 3, padding= 1, bias= False))
        
    def forward(self,x):
        return self.concat(x)
    
    def concat(self,x):
        return torch.cat([x, self.bottleneck(x)],1)

In [7]:
model = Bottleneck(3,32)
x =  model(test_image)
x.shape

torch.Size([1, 35, 224, 224])

In [8]:
class TransitionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionBlock, self).__init__()
        
        self.transition = nn.Sequential(nn.BatchNorm2d(in_channels), 
                              nn.ReLU(inplace= True),
                              nn.Conv2d(in_channels= in_channels, out_channels= out_channels,
                                                  kernel_size= 1, stride= 1,bias= False),
                              nn.AvgPool2d(kernel_size= 3, stride= 2, padding=0))
    def forward(self, x):
        return self.transition(x)

In [9]:
model1 = TransitionBlock(35,18) 
x1 =  model1(x)
x1.shape

torch.Size([1, 18, 111, 111])

In [258]:
class DenseBlock(nn.Module):
    def __init__(self,inner_channels, n_layers,growth_rate):
        super(DenseBlock,self).__init__()
        
        self.growth_rate = growth_rate
        self.n_layers = n_layers
        self.inner_channels = inner_channels 
        
        self.denseblock = self._make_layer(self.n_layers,self.growth_rate)
        
    def forward(self,x):
        return self.denseblock(x)
    
    
    def _make_layer(self,n_layers, growth_rate):
        
        layers = []

        
        for i in range(self.n_layers):
            layers.append(Bottleneck(in_channels= self.inner_channels, growth_rate = self.growth_rate))
            self.inner_channels += self.growth_rate
        
        
        return nn.Sequential(*layers)

In [297]:
class DenseNet(nn.Module):
    def __init__(self, config ,growth_rate= 32, image_channels= 3, compression= 0.5, 
                 clf= True, output_dim= 1000):
        super(DenseNet, self).__init__()
        
        n_layers = config
        #assert len(n_layers) == 4
        
        self.image_channels = image_channels
        self.output_dim = output_dim
        self.clf = clf
        self.growth_rate = growth_rate
        self.compression = compression
        
        
        self.inner_channels = 2 * self.growth_rate
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels= self.image_channels, 
                                             out_channels= self.inner_channels,kernel_size= 7, stride=2, 
                                             padding=3, bias= False),
                                   nn.BatchNorm2d(self.inner_channels),
                                   nn.ReLU(inplace= True))
        self.max_pool = nn.MaxPool2d(kernel_size= 3, stride=2, padding=1, dilation=1)
        
        self.block1 = DenseBlock(self.inner_channels,n_layers[0],self.growth_rate)
        self.inner_channels = self.block1.inner_channels
        out_channels= int(self.inner_channels * self.compression)
        self.transition1 = TransitionBlock(self.inner_channels, out_channels)
        self.inner_channels = out_channels
        
        self.block2 = DenseBlock(self.inner_channels,n_layers[1],self.growth_rate)
        self.inner_channels = self.block2.inner_channels
        out_channels= int(self.inner_channels * self.compression)
        self.transition2 = TransitionBlock(self.inner_channels, out_channels)
        self.inner_channels = out_channels
        
        self.block3 = DenseBlock(self.inner_channels,n_layers[2],self.growth_rate)
        self.inner_channels = self.block3.inner_channels
        out_channels= int(self.inner_channels * self.compression)
        self.transition3 = TransitionBlock(self.inner_channels, out_channels)
        self.inner_channels = out_channels

        self.block4 = DenseBlock(self.inner_channels,n_layers[3],self.growth_rate)        
        self.inner_channels = self.block4.inner_channels
        
        self.norm5 = nn.BatchNorm2d(self.inner_channels)
        self.averag_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.inner_channels, self.output_dim, bias=True)
                
        

        
        
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.block1(x)
        x = self.transition1(x)
        x = self.block2(x)
        x = self.transition2(x)
        x = self.block3(x)
        x = self.transition3(x)
        x = self.block4(x)
        x = self.norm5(x)
        x = self.averag_pool(x)
        if self.clf:
            x = x.view(x.shape[0], -1)
            x = self.fc(x)
    
        return x
        

In [321]:
densenet121_config = [6,12,24,16]

densenet169_config = [6,12,32,32]

densenet201_config = [6,12,48,32]

densenet264_config = [6,12,64,48]



In [322]:
model = DenseNet(config=densenet201_config)
model(test_image).shape

torch.Size([1, 1000])

In [320]:
def densenet121():
    return DenseNet(config=densenet121_config)

def densenet169():
    return DenseNet(config=densenet169_config)

def densenet201():
    return DenseNet(config=densenet201_config)

def densenet264():
    return DenseNet(config=densenet264_config)
    

In [324]:
model = densenet264()
model

DenseNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (block1): DenseBlock(
    (denseblock): Sequential(
      (0): Bottleneck(
        (bottleneck): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU(inplace=True)
          (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (1): Bottleneck(
        (bottleneck): Sequential(
          (0): BatchNorm2d(96, eps=1e-05, momentum=0.1

In [314]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,013,928 trainable parameters


In [315]:
model

DenseNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (block1): DenseBlock(
    (denseblock): Sequential(
      (0): Bottleneck(
        (bottleneck): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU(inplace=True)
          (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (1): Bottleneck(
        (bottleneck): Sequential(
          (0): BatchNorm2d(96, eps=1e-05, momentum=0.1

In [316]:
from torchvision.models import densenet201

In [317]:
model = densenet201()
model.children

<bound method Module.children of DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_

In [318]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,013,928 trainable parameters


In [319]:
20013928-20013928

0

In [228]:
3264/3

1088.0

In [45]:
def count_parameters(model1):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 8,163,240 trainable parameters


In [125]:
224*4*2

1792

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate= 32, image_channels= 3, compression= 0.5, 
                 clf= True, output_dim= 1000):
        super(DenseNet, self).__init__()
        
        self.image_channels = image_channels
        self.output_dim = output_dim
        self.clf = clf
        self.growth_rate = growth_rate
        self.compression = compression
        
        self.inner_channels = 2 * growth_rate
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels= self.image_channels, 
                                             out_channels= self.inner_channels,kernel_size= 7, stride=2, 
                                             padding=3),
                                   nn.BatchNorm2d(self.inner_channels),
                                   nn.ReLU(inplace= True))
        self.max_pool = nn.MaxPool2d(kernel_size= 3, stride=2, padding=1)
        
        self.layer1 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer2 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer3 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer4 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer5 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer6 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.transition1 = Transition(self.inner_channels, out_channels= int(self.inner_channels * self.compression))
        
        self.inner_channels =int(self.inner_channels * self.compression)
        
        self.layer7 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer8 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer9 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer10 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer11 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer12 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer13 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer14 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer15 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer16 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer17 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer18 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.transition2 = Transition(self.inner_channels, out_channels= int(self.inner_channels * self.compression))
        
        self.inner_channels =int(self.inner_channels * self.compression)
        
        self.layer19 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer20 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer21 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer22 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer23 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer24 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer25 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer26 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer27 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer28 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer29 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer30 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer31 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer32 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer33 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer34 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer35 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer36 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer37 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer38 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer39 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer40 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer41 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer42 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.transition3 = Transition(self.inner_channels, out_channels= int(self.inner_channels * self.compression))
        
        self.inner_channels =int(self.inner_channels * self.compression)
        

        
        self.layer43 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer44 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer45 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer46 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer47 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer48 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer49 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer50 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer51 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer52 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer53 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer54 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer55 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer56 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer57 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.layer58 = Bottleneck(in_channels=self.inner_channels,growth_rate= self.growth_rate)
        self.inner_channels += self.growth_rate
        
        self.averag_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.inner_channels, self.output_dim)
        
        
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.transition1(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layer13(x)
        x = self.layer14(x)
        x = self.layer15(x)
        x = self.layer16(x)
        x = self.layer17(x)
        x = self.layer18(x)
        x = self.transition2(x)
        x = self.layer19(x)
        x = self.layer20(x)
        x = self.layer21(x)
        x = self.layer22(x)
        x = self.layer23(x)
        x = self.layer24(x)
        x = self.layer25(x)
        x = self.layer26(x)
        x = self.layer27(x)
        x = self.layer28(x)
        x = self.layer29(x)
        x = self.layer30(x)
        x = self.layer31(x)
        x = self.layer32(x)
        x = self.layer33(x)
        x = self.layer34(x)
        x = self.layer35(x)
        x = self.layer36(x)
        x = self.layer37(x)
        x = self.layer38(x)
        x = self.layer39(x)
        x = self.layer40(x)
        x = self.layer41(x)
        x = self.layer42(x)
        x = self.transition3(x)
        x = self.layer43(x)
        x = self.layer44(x)
        x = self.layer45(x)
        x = self.layer46(x)
        x = self.layer47(x)
        x = self.layer48(x)
        x = self.layer49(x)
        x = self.layer50(x)
        x = self.layer51(x)
        x = self.layer52(x)
        x = self.layer53(x)
        x = self.layer54(x)
        x = self.layer55(x)
        x = self.layer56(x)
        x = self.layer57(x)
        x = self.layer58(x)
        x = self.averag_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
    
        return x
        